<a href="https://colab.research.google.com/github/poojamahajan0712/Data-Science-Portfolio/blob/main/PySpark/Ecommerce_Sales_data_analysis/TransactionData_exploratory_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
spark = SparkSession.builder.appName("Customer Segmentation")\
                              .config("spark.memory.offHeap.enabled","true")\
                              .config("spark.memory.offHeap.size","10g").getOrCreate()

#Specifies the size of the off-heap memory to 10 gigabytes. This memory is used for caching and computation.

In [4]:
df = spark.read.csv('/content/drive/MyDrive/AI Projects/Datasets/Amazon Sale Report/online_retail_listing.csv',header=True,sep=';')

In [5]:
df.show()

+-------+---------+--------------------+--------+---------------+-----+-----------+--------------+
|Invoice|StockCode|         Description|Quantity|    InvoiceDate|Price|Customer ID|       Country|
+-------+---------+--------------------+--------+---------------+-----+-----------+--------------+
| 489434|    85048|15CM CHRISTMAS GL...|      12|1.12.2009 07:45| 6,95|      13085|United Kingdom|
| 489434|   79323P|  PINK CHERRY LIGHTS|      12|1.12.2009 07:45| 6,75|      13085|United Kingdom|
| 489434|   79323W| WHITE CHERRY LIGHTS|      12|1.12.2009 07:45| 6,75|      13085|United Kingdom|
| 489434|    22041|"RECORD FRAME 7""...|      48|1.12.2009 07:45|  2,1|      13085|United Kingdom|
| 489434|    21232|STRAWBERRY CERAMI...|      24|1.12.2009 07:45| 1,25|      13085|United Kingdom|
| 489434|    22064|PINK DOUGHNUT TRI...|      24|1.12.2009 07:45| 1,65|      13085|United Kingdom|
| 489434|    21871| SAVE THE PLANET MUG|      24|1.12.2009 07:45| 1,25|      13085|United Kingdom|
| 489434| 

In [6]:
df.count(), len(df.columns) ## 1M rows

(1048575, 8)


* customer base
* total number of transactions
* Items extent in the transaction database

In [7]:
print("unique customers",df.select('Customer ID').distinct().count())
print("number of transcations",df.select('Invoice').distinct().count())## 52.9k transactions
print("unique items ",df.select("StockCode").distinct().count()) ## data is about buying of around 5.3k products.

unique customers 5925
number of transcations 52961
unique items  5304


* which countries are purchasing most
* how many countries available
* average transaction size
* average transaction value

In [8]:
df.dtypes

[('Invoice', 'string'),
 ('StockCode', 'string'),
 ('Description', 'string'),
 ('Quantity', 'string'),
 ('InvoiceDate', 'string'),
 ('Price', 'string'),
 ('Customer ID', 'string'),
 ('Country', 'string')]

In [9]:
df.select('Country').distinct().count()

43

In [10]:
#90% customers are from UK
df.groupby('Country').agg(countDistinct('Customer ID')\
                      .alias('customer_count'))\
                      .orderBy('customer_count',ascending=False)\
                      .show()

+---------------+--------------+
|        Country|customer_count|
+---------------+--------------+
| United Kingdom|          5397|
|        Germany|           106|
|         France|            94|
|          Spain|            40|
|        Belgium|            29|
|       Portugal|            24|
|    Netherlands|            23|
|    Switzerland|            22|
|         Sweden|            19|
|          Italy|            17|
|        Finland|            15|
|      Australia|            15|
|Channel Islands|            14|
|         Norway|            13|
|        Austria|            13|
|         Cyprus|            11|
|        Denmark|            11|
|          Japan|            10|
|            USA|             9|
|    Unspecified|             7|
+---------------+--------------+
only showing top 20 rows



In [11]:
df.dtypes

[('Invoice', 'string'),
 ('StockCode', 'string'),
 ('Description', 'string'),
 ('Quantity', 'string'),
 ('InvoiceDate', 'string'),
 ('Price', 'string'),
 ('Customer ID', 'string'),
 ('Country', 'string')]

In [12]:
df = df.withColumn('qty',col('Quantity').cast('float'))
df.groupby('Invoice').agg(sum('qty')\
                      .alias('basket_size'))\
                      .select(median('basket_size')).show()

+-------------------+
|median(basket_size)|
+-------------------+
|               93.0|
+-------------------+



In [13]:

df = df.withColumn("price_edited",regexp_replace(col('Price'),',','.').cast('float'))
df.show(5)

+-------+---------+--------------------+--------+---------------+-----+-----------+--------------+----+------------+
|Invoice|StockCode|         Description|Quantity|    InvoiceDate|Price|Customer ID|       Country| qty|price_edited|
+-------+---------+--------------------+--------+---------------+-----+-----------+--------------+----+------------+
| 489434|    85048|15CM CHRISTMAS GL...|      12|1.12.2009 07:45| 6,95|      13085|United Kingdom|12.0|        6.95|
| 489434|   79323P|  PINK CHERRY LIGHTS|      12|1.12.2009 07:45| 6,75|      13085|United Kingdom|12.0|        6.75|
| 489434|   79323W| WHITE CHERRY LIGHTS|      12|1.12.2009 07:45| 6,75|      13085|United Kingdom|12.0|        6.75|
| 489434|    22041|"RECORD FRAME 7""...|      48|1.12.2009 07:45|  2,1|      13085|United Kingdom|48.0|         2.1|
| 489434|    21232|STRAWBERRY CERAMI...|      24|1.12.2009 07:45| 1,25|      13085|United Kingdom|24.0|        1.25|
+-------+---------+--------------------+--------+---------------

In [14]:
df.dtypes

[('Invoice', 'string'),
 ('StockCode', 'string'),
 ('Description', 'string'),
 ('Quantity', 'string'),
 ('InvoiceDate', 'string'),
 ('Price', 'string'),
 ('Customer ID', 'string'),
 ('Country', 'string'),
 ('qty', 'float'),
 ('price_edited', 'float')]

In [15]:
df.groupby('Invoice').agg(sum('price_edited')\
                      .alias('avg_ord_val'))\
                      .select(mean('avg_ord_val')).show()

+-----------------+
| avg(avg_ord_val)|
+-----------------+
|91.61683989018914|
+-----------------+



* rows with negative price, quantity
* rows with null customer_id

In [16]:
# rows with negative price
df.filter(col('price_edited')<0).show()

+-------+---------+---------------+--------+----------------+---------+-----------+--------------+---+------------+
|Invoice|StockCode|    Description|Quantity|     InvoiceDate|    Price|Customer ID|       Country|qty|price_edited|
+-------+---------+---------------+--------+----------------+---------+-----------+--------------+---+------------+
|A506401|        B|Adjust bad debt|       1|29.04.2010 13:36|-53594,36|       NULL|United Kingdom|1.0|   -53594.36|
|A516228|        B|Adjust bad debt|       1|19.07.2010 11:24|-44031,79|       NULL|United Kingdom|1.0|   -44031.79|
|A528059|        B|Adjust bad debt|       1|20.10.2010 12:04|-38925,87|       NULL|United Kingdom|1.0|   -38925.87|
|A563186|        B|Adjust bad debt|       1|12.08.2011 14:51|-11062,06|       NULL|United Kingdom|1.0|   -11062.06|
|A563187|        B|Adjust bad debt|       1|12.08.2011 14:52|-11062,06|       NULL|United Kingdom|1.0|   -11062.06|
+-------+---------+---------------+--------+----------------+---------+-

In [17]:
print("rows with negative quantity",df.filter(col('qty')<0).count())
df.filter(col('qty')<0).show()
## all have invoice staring with C

rows with negative quantity 22697
+-------+---------+--------------------+--------+---------------+-----+-----------+--------------+-----+------------+
|Invoice|StockCode|         Description|Quantity|    InvoiceDate|Price|Customer ID|       Country|  qty|price_edited|
+-------+---------+--------------------+--------+---------------+-----+-----------+--------------+-----+------------+
|C489449|    22087|PAPER BUNTING WHI...|     -12|1.12.2009 10:33| 2,95|      16321|     Australia|-12.0|        2.95|
|C489449|   85206A|CREAM FELT EASTER...|      -6|1.12.2009 10:33| 1,65|      16321|     Australia| -6.0|        1.65|
|C489449|    21895|POTTING SHED SOW ...|      -4|1.12.2009 10:33| 4,25|      16321|     Australia| -4.0|        4.25|
|C489449|    21896|  POTTING SHED TWINE|      -6|1.12.2009 10:33|  2,1|      16321|     Australia| -6.0|         2.1|
|C489449|    22083|PAPER CHAIN KIT R...|     -12|1.12.2009 10:33| 2,95|      16321|     Australia|-12.0|        2.95|
|C489449|    21871| SA

In [18]:
df.filter(col('Description')=='Adjust bad debt').count()

6

In [19]:
# rows with null customer_id
print("count of rows with null customer id",df.filter(col('Customer ID').isNull()).count())
df.filter(col('Customer ID').isNull()).show()

count of rows with null customer id 236682
+-------+---------+--------------------+--------+---------------+-----+-----------+--------------+------+------------+
|Invoice|StockCode|         Description|Quantity|    InvoiceDate|Price|Customer ID|       Country|   qty|price_edited|
+-------+---------+--------------------+--------+---------------+-----+-----------+--------------+------+------------+
| 489464|    21733|        85123a mixed|     -96|1.12.2009 10:52|    0|       NULL|United Kingdom| -96.0|         0.0|
| 489463|    71477|               short|    -240|1.12.2009 10:52|    0|       NULL|United Kingdom|-240.0|         0.0|
| 489467|   85123A|         21733 mixed|    -192|1.12.2009 10:53|    0|       NULL|United Kingdom|-192.0|         0.0|
| 489521|    21646|                NULL|     -50|1.12.2009 11:44|    0|       NULL|United Kingdom| -50.0|         0.0|
| 489525|   85226C|BLUE PULL BACK RA...|       1|1.12.2009 11:49| 0,55|       NULL|United Kingdom|   1.0|        0.55|
| 489

In [20]:
df.filter(col('Invoice').like('C%')).count()

19261

In [21]:
df.filter((col('qty')<0) & (~col('Invoice').like('C%'))).show() #3.4k

+-------+---------+---------------+--------+---------------+-----+-----------+--------------+-------+------------+
|Invoice|StockCode|    Description|Quantity|    InvoiceDate|Price|Customer ID|       Country|    qty|price_edited|
+-------+---------+---------------+--------+---------------+-----+-----------+--------------+-------+------------+
| 489464|    21733|   85123a mixed|     -96|1.12.2009 10:52|    0|       NULL|United Kingdom|  -96.0|         0.0|
| 489463|    71477|          short|    -240|1.12.2009 10:52|    0|       NULL|United Kingdom| -240.0|         0.0|
| 489467|   85123A|    21733 mixed|    -192|1.12.2009 10:53|    0|       NULL|United Kingdom| -192.0|         0.0|
| 489521|    21646|           NULL|     -50|1.12.2009 11:44|    0|       NULL|United Kingdom|  -50.0|         0.0|
| 489655|    20683|           NULL|     -44|1.12.2009 17:26|    0|       NULL|United Kingdom|  -44.0|         0.0|
| 489660|    35956|           lost|   -1043|1.12.2009 17:43|    0|       NULL|Un

* Min date, max date


In [22]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")#- need to set this , otherwise throws error
df = df.withColumn('date',to_timestamp(col('InvoiceDate'),"dd.MM.yyyy HH:mm"))

In [23]:
df.select(min('date')).show(), df.select(max('date')).show()

+-------------------+
|          min(date)|
+-------------------+
|2009-12-01 07:45:00|
+-------------------+

+-------------------+
|          max(date)|
+-------------------+
|2011-12-04 13:15:00|
+-------------------+



(None, None)

* month_year with highest number of transactions - extract month and aggregate
* top 5 items bought by customers (top based on count)



In [34]:
df = df.withColumn('month_year',concat(month(col('date')),lit("_"),year(col('date'))))
df.groupby('month_year').agg(countDistinct('Invoice')\
                        .alias("num_transaction"))\
                        .orderBy("num_transaction",ascending=False).show(10)
# max transactions bought in month_year nov 2010, nov 2011, oct 2010


+----------+---------------+
|month_year|num_transaction|
+----------+---------------+
|   11_2010|           3669|
|   11_2011|           3462|
|   10_2010|           2965|
|   10_2011|           2637|
|    5_2010|           2418|
|    9_2010|           2375|
|    3_2010|           2367|
|   12_2009|           2330|
|    9_2011|           2327|
|    6_2010|           2216|
+----------+---------------+
only showing top 10 rows



In [35]:
df = df.withColumn('month',month(col('date')))
df.groupby('month').agg(countDistinct('Invoice')\
                        .alias("num_transaction_month"))\
                        .orderBy("num_transaction_month",ascending=False).show()
# max transactions bought in month last quarter of the year- holiday season (thanksgiving, etc.)

+-----+---------------------+
|month|num_transaction_month|
+-----+---------------------+
|   11|                 7131|
|   10|                 5602|
|   12|                 4703|
|    9|                 4702|
|    5|                 4580|
|    3|                 4350|
|    6|                 4228|
|    7|                 3944|
|    4|                 3636|
|    8|                 3614|
|    2|                 3362|
|    1|                 3109|
+-----+---------------------+



In [45]:
aggregated_df = df.groupby('StockCode','Description').agg(sum('qty')\
                                                      .alias('total_count'))\
                                                      .orderBy('total_count',ascending = False)
aggregated_df.show(20, truncate=False)

## Cake cases are hit

+---------+-----------------------------------+-----------+
|StockCode|Description                        |total_count|
+---------+-----------------------------------+-----------+
|84077    |WORLD WAR 2 GLIDERS ASSTD DESIGNS  |107472.0   |
|85123A   |WHITE HANGING HEART T-LIGHT HOLDER |91863.0    |
|84879    |ASSORTED COLOUR BIRD ORNAMENT      |80608.0    |
|85099B   |JUMBO BAG RED RETROSPOT            |77222.0    |
|17003    |BROCADE RING PURSE                 |70423.0    |
|21977    |PACK OF 60 PINK PAISLEY CAKE CASES |56366.0    |
|84991    |60 TEATIME FAIRY CAKE CASES        |54271.0    |
|22197    |SMALL POPCORN HOLDER               |49616.0    |
|21212    |PACK OF 72 RETROSPOT CAKE CASES    |49030.0    |
|21212    |PACK OF 72 RETRO SPOT CAKE CASES   |46106.0    |
|22492    |MINI PAINT SET VINTAGE             |43904.0    |
|15036    |ASSORTED COLOURS SILK FAN          |43542.0    |
|21213    |PACK OF 72 SKULL CAKE CASES        |40534.0    |
|84755    |COLOUR GLASS T-LIGHT HOLDER H

* filter data removing invoice, and their c counterparts conisdering as cancelled
* number of customers responsible for 80% of sales- calc from filtered data
* create Recency, frequency and monetary flags

In [25]:
# https://www.datacamp.com/tutorial/pyspark-tutorial-getting-started-with-pyspark
# https://www.kaggle.com/datasets/thedevastator/unlock-profits-with-e-commerce-sales-data

# other dataset
# https://www.kaggle.com/code/toludoyinshopein/rfm-segmentation-with-pyspark


## load dataset
## create spark session
## data exploration
## data preprocessing
## feature engg
## model building